# Hybrid Monte Carlo

## Monte Carlo Simulation

In this notebook we illustrate the Monte Carlo simulation framework. This includes is structured along the following steps:

  1.  Setting up a model
  2.  Contructing and running a simulation
  3.  Inspecting the simulation
  4.  Calculate future modeled quantities

We use a couple of standard packages for calculation and analysis

In [ ]:
import sys
sys.path.append('../')  # make python find our modules
import numpy as np
import pandas as pd
import plotly.express as px

The following auxilliary method lists the relevant members of an object. We use it to inspect the objects created. 

In [ ]:
def members(obj):
    return [f for f in dir(obj) if not f.startswith('_')] 

The starting point for modelling is a discount curve. For now we use a flat forward furve. More advanced curve specifications can easily be included e.g. via QuantLib. Our modelling framework only requires that a yieldcurve provides a method *discunt(t)* for a time parameter *t*.

In [ ]:
from src.termstructures.YieldCurve import YieldCurve
discCurve = YieldCurve(rate=0.03)
display(members(discCurve))

### Setting up a model

For this example we use a Hull White interest rate model. This type of model will typically also be the building block for more complex models.

The Hull White model allows for piece-wise constant short rate volatility and constant mean reversion.

In [ ]:
from src.models.HullWhiteModel import HullWhiteModel
times = np.array([ 2.0,    5.0,    10.0    ])
vols  = np.array([ 0.0050, 0.0060,  0.0070 ])
mean  = 0.03
#
model = HullWhiteModel(discCurve,mean,times,vols)
display(members(model))

### Contructing and running a simulation

For a simulation we need to specify a time grid on which model states are evolved, number of Monte Carlo paths and a (pseudo) random number seed.

The simulation is run at construction of the object.

In [ ]:
from src.simulations.MCSimulation import MCSimulation
seed = 314159265359
nPaths = 3
times = np.linspace(0.0, 10.0, 11)
display(times)

In [ ]:
mcsim = MCSimulation(model,times,nPaths,seed)
display(members(mcsim))

### Inspecting the simulation

The Monte Carlo simulation calculates independent standard normal variables *dW*. The data are stored as 3-dimensional array *dW\[nPaths,times-1,factor\]*.

In [ ]:
display(mcsim.dW.shape)

The Hull White model is a 1-factor model. Thus last dimension is 1. We can verify this by asking the model for its random factors.



In [ ]:
display(model.factorAliases())

We illustrate random samples per time step and path.

In [ ]:
# we prep the dW's for nice plotting
data = pd.DataFrame(mcsim.dW[:,:,0]).T
data.columns = [str(c) for c in data.columns]
ts = pd.Series(times[:-1],name='times')
data = pd.concat([ts,data],axis=1)
data = pd.melt(data,id_vars='times', value_vars=list(data.columns[1:]),var_name='path',value_name='dW')
# 
fig = px.scatter(data,x='times',y='dW',color='path')
fig.show()

The core functionality of Monte Carlo simulation is simulating state variables *X*. This is implemented based on the model specification and the model's *evolve()* method. State variables are stored in a 3-dimensional array *X\[nPaths,times,states\]*.

In [ ]:
display(mcsim.X.shape)

We simulate the Hull White state variable *x(t)=r(t)-f(0,t)*. However, for derivative pricing we also need the numeraire (here bank account), $B(t) = P(0,t)^{-1}\exp\left\{\int_0^t x(s)sd\right\}$. Thus, we store the integrated state variable $s(t)=\int_0^t x(s)ds$ in the second component of the Monte Carlo state.

We can verify the model states by asking the model for its internal representation.

In [ ]:
display(model.stateAliases())

In [ ]:
data_x = pd.DataFrame(mcsim.X[:,:,0]).T
data_x.columns = ['x_'+str(c) for c in data_x.columns]
data_s = pd.DataFrame(mcsim.X[:,:,1]).T
data_s.columns = ['s_'+str(c) for c in data_s.columns]
ts = pd.Series(times,name='times')
data = pd.concat([ts,data_x,data_s], axis=1)
data = pd.melt(data,id_vars='times', value_vars=list(data.columns[1:]),var_name='path',value_name='X')
# 
fig = px.scatter(data,x='times',y='X',color='path')
fig.show()

### Calculate future modeled quantities

With the Monte Carlo state(s) we can now calculate simulated future model quantities. For an interest rate model these quantities are zero coupon bonds $P(t,T)=P(x;t,T)$.

In [ ]:
k = 10  # we pick the last simulated time
t = times[k]
T = t + 5.0  # and 5y zero bond maturity
display((t,T))

The simulates Monte Carlo states:

In [ ]:
states = mcsim.X[:,k,:]
display(states)

And resulting simulated zero coupon bonds:

In [ ]:
zeroBonds = np.array([ model.zeroBond(t,T,X,None) for X in states ])
display(zeroBonds)

We can compare simulated zero coupon bonds to forward zero bonds $P(0,T)/P(0,t)$ calculated from the initial discount curve.

In [ ]:
zeroBondFwd = discCurve.discount(T)/discCurve.discount(t)
display(zeroBondFwd)

However, we need to change measure. Hull White model simulation is in risk-neutral measure. Forward zero bond is a martingale in the *T*-forward measure.

In [ ]:
numeraires = np.array([ model.numeraire(t,X) for X in states ])
zeroBonds_T = zeroBonds / numeraires / discCurve.discount(t)
display(zeroBonds_T)

In [ ]:
zeroBonds_T_av = np.average(zeroBonds_T)
display(zeroBonds_T_av)

### Complex model setup

In the example above we only modelled inerest rates for a sngle currency with a Hull White model. For hybrid modelling we typically need a model for various components.

In this section we give and example for a hybrid model. This should demonstrate the overall principle without going into the details of the math for now.

A complex model is composed of individual models which are *plugged together* in a sensible way. We consider the following component models:

  -  Rates model for foreign and domestic curves (here Hull White model)
  -  Discounted cash flow (DCF) model for dividend yields
  -  Asset model for FX rates and single stocks or indices
  -  Stochastic intensity (or hazard rate) model for credit modelling
  -  Stochastic funding or discount spread model

We start with our base currency rates model and re-use our Hull White model from above.

In [ ]:
domRates = model

Next we specify a FX model for EUR/USD. Since we already defined our base (or domestic) currency to be EUR we can not directly model EUR/USD. Instead we model the inverse FX spot USD/EUR where USD is foreign currency and EUR is domestic currency. To make things precise, we model the EUR-price of one unit of USD currency.

For FX modelling we need an *AssetModel*. For now our asset modelling is very simple. We model lognormal underlyings with constant volatility $\sigma$. And of course, we need to specify our initial FX rate (here *X0*).

In [ ]:
from src.models.AssetModel import AssetModel
usdAsset = AssetModel(X0=0.85,sigma=0.15)

For foreign rates modelling we also use a Hull White model. Model parameters are kept simple for now.

In [ ]:
usdRates = HullWhiteModel(YieldCurve(0.01), 0.01, np.array([10.0]), np.array([0.0050]))

We also want to model an equity index, say Euro Stoxx 50. This is also realised via an *AssetModel*. Now it represents the index price (instead of the FX rate above).

In [ ]:
sx5eAsset = AssetModel(X0=3200.0,sigma=0.1)

Dividend yields are modelled as deterministic. For deterministic (rates) modelling we have a *DcfModel*.

In [ ]:
from src.models.DeterministicModel import DcfModel
sx5eYield = DcfModel(YieldCurve(0.015))

Now we can plug things together into a *HybridModel*. Here we also specify *aliases* to keep track of what we are modelling.

In [ ]:
from src.models.HybridModel import HybridModel
hybModel = HybridModel(domAlias='EUR',domRatesModel=domRates,
    forAliases     = [    'USD',    'SX5E' ],
    forAssetModels = [ usdAsset, sx5eAsset ],
    forRatesModels = [ usdRates, sx5eYield ], correlations=None)

Note that we did not specify a correlation matrix (which is very important for hybrid modelling). We can inspect the hybrid model to double-check the size of the required correlaction.

In [ ]:
display(hybModel.factorAliases())

We see, we have four factors. To keep things simple, we assume zero correlation and thus may use the identity matrix.

In [ ]:
hybModel.correlations = np.identity(hybModel.factors())

We also want to model credit risk with respect to a counterparty, say GS. Credit spreads are also modelled via rates models. To keep things brief we use a Hull White model again.

In [ ]:
gsCredit = HullWhiteModel(YieldCurve(0.0050), 0.01, np.array([10.0]), np.array([0.0050]))

Credit components are attached to rates or hybrid models by means of a *CreditModel*.

In [ ]:
from src.models.CreditModel import CreditModel
crModel = CreditModel(baseModel=hybModel,creditAliases=['GS'],
    creditModels=[gsCredit],correlations=None)

Finally, to make things complete we add an additional stochastic discount spread to mimic stochastic funding costs. Again we use a Hull White model for the spread component.

In [ ]:
fundRates = HullWhiteModel(YieldCurve(0.0025), 0.01, np.array([10.0]), np.array([0.0030]))

In [ ]:
from src.models.SpreadModel import SpreadModel
sprdModel = SpreadModel(baseModel=crModel,sprdModel=fundRates,correlations=None)

Now we got everything in one place. How does the final model look like?

We can check the all the model factors.

In [ ]:
display(sprdModel.factorAliases())

This representation also helps to set up a corresponding correlation matrix.

Note that we do not have a stochastic factor for SX5E dividends eventhough we specified a model component *sx5eYield*. This is because dividend yields are assumed deterministic and *DcfModel* does not have stochastic factors (and states).

Similarly, we can ask the model for all the modelled states.

In [ ]:
display(sprdModel.stateAliases())

With the complex model at hand we can now run Monte Carlo simulation again. Nothing changes here compared to the basic Hull Whte model.

In [ ]:
seed = 314159265359
nPaths = 1
times = np.linspace(0.0, 10.0, 3)
mcsim = MCSimulation(sprdModel,times,nPaths,seed)
display(mcsim.X[0,:,:])

Now we can ask the model again for its simulated quantities.

In [ ]:
t = 10.0
X = mcsim.X[0,2,:]

We list a few examples here

In [ ]:
# Domestic currency zero coupon bond without stochastic funding component
display(sprdModel.zeroBond(t,t+5.0,X,'EUR'))
# Domestic currency zero coupon bond with stochastic funding component
display(sprdModel.zeroBond(t,t+5.0,X,None))
# foreign zero coupon bond
display(sprdModel.zeroBond(t,t+5.0,X,'USD'))
# USD/EUR
display(sprdModel.asset(t,X,'USD'))
# Euro Stoxx 50
display(sprdModel.asset(t,X,'SX5E'))
# GS hazard rate
display(sprdModel.hazardRate(t,X,'GS'))


